In [1]:
import requests  # For making HTTP requests
from bs4 import BeautifulSoup  # For web scraping
import re
import pandas as pd
import zipfile as zip

In [2]:
df = pd.read_csv('zip_price.tsv.gz', compression='gzip', sep='\t')

In [3]:
df

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2022-06-01,2022-08-31,90,zip code,2,19125,f,Zip Code: 45390,NaN,Ohio,...,0.266667,NaN,NaN,NaN,0.428571,-0.238095,0.228571,"Greenville, OH",24820,2025-02-10 14:21:52
1,2021-09-01,2021-11-30,90,zip code,2,38345,f,Zip Code: 92507,NaN,California,...,0.201081,NaN,NaN,NaN,0.457143,-0.011389,-0.067857,"Riverside, CA",40140,2025-02-10 14:21:52
2,2023-04-01,2023-06-30,90,zip code,2,34424,f,Zip Code: 79404,NaN,Texas,...,0.000000,NaN,NaN,NaN,0.210526,-0.083591,-0.075188,"Lubbock, TX",31180,2025-02-10 14:21:52
3,2021-03-01,2021-05-31,90,zip code,2,1738,f,Zip Code: 04969,NaN,Maine,...,0.222222,NaN,NaN,NaN,0.333333,0.083333,0.000000,"Bangor, ME",12620,2025-02-10 14:21:52
4,2022-06-01,2022-08-31,90,zip code,2,7527,f,Zip Code: 19403,NaN,Pennsylvania,...,0.033102,NaN,NaN,NaN,0.668639,-0.070250,-0.061594,"Montgomery County, PA",33874,2025-02-10 14:21:52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8749561,2022-07-01,2022-09-30,90,zip code,2,14248,f,Zip Code: 33569,NaN,Florida,...,-0.194464,NaN,NaN,NaN,0.517241,-0.073235,-0.241921,"Tampa, FL",45300,2025-02-10 14:21:52
8749562,2019-06-01,2019-08-31,90,zip code,2,35931,f,Zip Code: 84111,NaN,Utah,...,-0.086957,NaN,NaN,NaN,0.258065,-0.075269,-0.141935,"Salt Lake City, UT",41620,2025-02-10 14:21:52
8749563,2014-07-01,2014-09-30,90,zip code,2,14138,f,Zip Code: 33408,NaN,Florida,...,0.000000,NaN,NaN,NaN,0.000000,0.000000,-0.111111,"West Palm Beach, FL",48424,2025-02-10 14:21:52
8749564,2018-09-01,2018-11-30,90,zip code,2,31193,f,Zip Code: 72727,NaN,Arkansas,...,-0.015551,NaN,NaN,NaN,0.019608,-0.000392,-0.049358,"Fayetteville, AR",22220,2025-02-10 14:21:52


In [4]:
df = df[df['state'] == 'California']
df.columns

Index(['period_begin', 'period_end', 'period_duration', 'region_type',
       'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region',
       'city', 'state', 'state_code', 'property_type', 'property_type_id',
       'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy',
       'median_list_price', 'median_list_price_mom', 'median_list_price_yoy',
       'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf',
       'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold',
       'homes_sold_mom', 'homes_sold_yoy', 'pending_sales',
       'pending_sales_mom', 'pending_sales_yoy', 'new_listings',
       'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom',
       'inventory_yoy', 'months_of_supply', 'months_of_supply_mom',
       'months_of_supply_yoy', 'median_dom', 'median_dom_mom',
       'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom',
       'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom',
 

In [5]:
df['region']

1          Zip Code: 92507
7          Zip Code: 91948
30         Zip Code: 93925
35         Zip Code: 95519
37         Zip Code: 93041
                ...       
8749494    Zip Code: 92683
8749516    Zip Code: 94501
8749532    Zip Code: 92011
8749555    Zip Code: 93638
8749560    Zip Code: 92402
Name: region, Length: 800032, dtype: object

In [6]:
df['median_sale_price']

1          533000.0
7          175000.0
30         875000.0
35         357000.0
37         314000.0
             ...   
8749494    715000.0
8749516    811250.0
8749532    586500.0
8749555    200000.0
8749560    260000.0
Name: median_sale_price, Length: 800032, dtype: float64

In [7]:
df = df[['region', 'median_sale_price']]
df

,region,median_sale_price
1,Zip Code: 92507,533000.0
7,Zip Code: 91948,175000.0
30,Zip Code: 93925,875000.0
35,Zip Code: 95519,357000.0
37,Zip Code: 93041,314000.0
...,...,...
8749494,Zip Code: 92683,715000.0
8749516,Zip Code: 94501,811250.0
8749532,Zip Code: 92011,586500.0
8749555,Zip Code: 93638,200000.0


In [8]:
def extract_zip(zip):
    return zip[10:]

extract_zip('Zip Code: 92507')
df['region'] = df['region'].transform(extract_zip)

In [9]:
df.to_csv('data/price_per_zip.csv')

In [15]:
df.shape[0]

800032

In [30]:
import zipfile
zip_path = 'totalData_zestimate.csv.zip'

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open("totalData_zestimate.csv") as f:  # Change "yourfile.csv" to the actual filename
        data = pd.read_csv(f)

/var/folders/r6/j2xslqfj0lg0bxy1nj8b6fzm0000gn/T/ipykernel_98318/4027753126.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f)


In [21]:
df = df.groupby('region').median().reset_index()

In [32]:
data = pd.merge(data, df, left_on='PostalCode', right_on='region', how='left')

In [39]:
data = data.iloc[:, 1:]

In [41]:
data.to_csv('data/data_median_redfin.csv')